In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from TModel.TransformerModel import TranscriptionTransformerModel
from Tokenizer.loaderH5 import H5GuitarTokenizer
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
from TUtils import random_string
from lightning import Trainer
import torch
from TranscriptionDataset import TranscriptionDataset
torch.set_float32_matmul_precision('medium')

In [2]:
datasetLocation = "Trainsets/S_Tier_1695619803_mTokens400_mNoS5.hdf5"
wandbProject = "TranscriptionModel_Test"
batchSize = 4
num_workers = 2

In [3]:
dataset,pipe = TranscriptionDataset.getDataPipe(datasetLocation,batchSize)
# train_pipe,test_pipe = pipe.random_split({"train":0.8,"test":0.2},42)

train_dl = DataLoader(dataset=pipe,batch_size=None,num_workers=num_workers)
# test_dl = DataLoader(dataset=test_pipe, batch_size=batchSize,num_workers=num_workers)

In [4]:
model = TranscriptionTransformerModel(
    dataset.getVocabSize(),
    d_ff=2048,
    # embeddingCheckpoint="Models/GuitarToken/Max2Length.ckpt"
)

In [5]:
wandb_logger = WandbLogger(project=wandbProject)
wandb_logger.experiment.config.update(dataset.meta_data)
wandb_logger.experiment.config["batchSize"] = batchSize
checkpoint_callback = ModelCheckpoint(
    monitor='train_loss',
    dirpath=f'Models/GuitarTranscription/{random_string(10)}/',
    filename='GuitarTranscriptionModel-{epoch:02d}-{train_loss:.2f}',
    every_n_train_steps=1000,
    save_top_k=3,
    mode='min',
)
trainer = Trainer(
    default_root_dir='Models/',
    max_epochs=5,
    # profiler="simple",
    logger=wandb_logger,
    # callbacks=[checkpoint_callback],
    # max_time="00:00:05:00"
    precision="bf16-mixed",    
)

wandb: Currently logged in as: draguve. Use `wandb login --relogin` to force relogin


Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model=model, train_dataloaders=train_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type               | Params
----------------------------------------------------------
0 | positionalEncoding | PositionalEncoding | 0     
1 | transformer_model  | Transformer        | 44.1 M
2 | tgt_embedding      | Embedding          | 3.1 M 
3 | outputNorm         | Linear             | 3.1 M 
4 | loss               | CrossEntropyLoss   | 0     
----------------------------------------------------------
50.4 M    Trainable params
0         Non-trainable params
50.4 M    Total params
201.703   Total estimated model params size (MB)
C:\Users\ritwi\anaconda3\envs\pytorch\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  r

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [7]:
trainer.save_checkpoint("Models/GuitarTranscription/5s400Tokens_1e-5/smallDataset5epoch.ckpt")